# EIA Crude Oil Data Analysis using Neural Network

This research is mainly to utilize EIA api to import data, and apply machine learning techniques to predict Crude Oil Futures price movement, using all factors associated with EIA

Example of API data was given in the link below (Put your API key in the YOUR_API_KEY_HERE area in order to obtain data): <br>
http://api.eia.gov/series/?api_key=YOUR_API_KEY_HERE&series_id=PET.WCRFPUS2.W

All data are given in this link (hit the key button will show API link): <br>
https://www.eia.gov/dnav/pet/pet_sum_sndw_dcus_nus_w.htm

In [150]:
import numpy as np
import pandas as pd
import requests
import statsmodels.api as sm # Apply Simple Multi-Linear Regression for analysis
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

We will import data using requests packages, we will first try on 3 key data below:<br>
Series ID = PET.WCRFPUS2.W #US Field Production of Crude Oil Weekly <br>
Series ID = PET.WCRRIUS2.W #US Refiner Net Input of Crude Oil<br>
Series ID = PET.WTTIMUS2.W #US Imports of Crude Oil and Petroleum Products, Weekly

In [89]:
api_key = "af034aec5e5242b2bc1ccaef0c93887c"
series_id = ['PET.WCRFPUS2.W', 'PET.WCRRIUS2.W', 'PET.WTTIMUS2.W']

df = pd.DataFrame() # Declare dataframe

for i in series_id:
    data = requests.get('http://api.eia.gov/series/?api_key=' + api_key + '&series_id=' + i).json()
    data = pd.DataFrame(data['series'])
    
    headers = ["time", i]
    tmp = data[['data']]
    tmp = tmp['data'][0]
    tmpdf = pd.DataFrame(tmp, columns = headers)
    df = pd.concat([df, tmpdf], axis = 1)


df = df.iloc[:, ~df.columns.duplicated()]
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values(by = ['time']) # Sort the dataframe ascending by time
df = df.dropna()

Below is how the dataframe looks like

In [90]:
df[0:5]

,time,PET.WCRFPUS2.W,PET.WCRRIUS2.W,PET.WTTIMUS2.W
1424,1991-02-08,7463.0,12973,6877.0
1423,1991-02-15,7427.0,12931,6573.0
1422,1991-02-22,7415.0,13107,6221.0
1421,1991-03-01,7404.0,13164,6188.0
1420,1991-03-08,7394.0,13082,7127.0


### Obtaining Historical price data from Quandl api:
https://www.quandl.com/api/v3/datasets/CHRIS/CME_CL1

In [91]:
data = requests.get('https://www.quandl.com/api/v3/datasets/CHRIS/CME_CL1').json()

keys = list(data['dataset'].keys())
data = data['dataset']
price = data['data']
headers = data['column_names']
price_df = pd.DataFrame(price, columns = headers)
price_df = price_df.rename(columns = {'Date':'time'}) # Change Date column name into time to be consistent
price_df['time'] = pd.to_datetime(price_df['time'])
price_df = price_df.sort_values(by = ['time']) # Sort the dataframe ascending by time
price_df[0:5]

,time,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
8837,1983-03-30,29.01,29.56,29.01,29.40,NaN,29.40,949.0,470.0
8836,1983-03-31,29.40,29.60,29.25,29.29,NaN,29.29,521.0,523.0
8835,1983-04-04,29.30,29.70,29.29,29.44,NaN,29.44,156.0,583.0
8834,1983-04-05,29.50,29.80,29.50,29.71,NaN,29.71,175.0,623.0
8833,1983-04-06,29.90,29.92,29.65,29.90,NaN,29.90,392.0,640.0


Merge Factors Data and Historical pricing data

In [92]:
df = df.merge(price_df, left_on = 'time', right_on = 'time', how = 'left')
df = df.drop(columns = ['Change', 'Open', 'High', 'Low', 'Settle'])
df = df.dropna()

In [93]:
# Calculate Changes in term of returns
df.set_index('time')
df['Last'] = df['Last'].shift(-1)
df.iloc[:, 1:len(df.columns)] = (df.iloc[:, 1:len(df.columns)] - df.iloc[:, 1:len(df.columns)].shift(1)) / df.iloc[:, 1:len(df.columns)].shift(1)
df = df.dropna()
df[0:5]

,time,PET.WCRFPUS2.W,PET.WCRRIUS2.W,PET.WTTIMUS2.W,Last,Volume,Previous Day Open Interest
1,1991-02-15,-0.004824,-0.003237,-0.044205,-0.142241,0.195222,-0.381740
2,1991-02-22,-0.001616,0.013611,-0.053552,0.082077,0.181597,0.572817
3,1991-03-01,-0.001483,0.004349,-0.005305,-0.003612,-0.521073,-0.107887
4,1991-03-08,-0.001351,-0.006229,0.151745,0.036251,0.256231,0.014883
5,1991-03-15,0.002434,-0.020257,-0.152378,0.004498,-0.296973,-0.229741


### Apply Multi-Linear-Regression

In [94]:
df.columns

Index(['time', 'PET.WCRFPUS2.W', 'PET.WCRRIUS2.W', 'PET.WTTIMUS2.W', 'Last',
       'Volume', 'Previous Day Open Interest'],
      dtype='object')

In [95]:
y = df['Last']
X = df[series_id]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Last   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.919
Date:                Sat, 02 Jun 2018   Prob (F-statistic):              0.125
Time:                        22:43:59   Log-Likelihood:                 2160.4
No. Observations:                1354   AIC:                            -4313.
Df Residuals:                    1350   BIC:                            -4292.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.0022      0.001      1.648      0.100      -0.000       0.005
PET.WCRFPUS2.W     0.0693      0.068      1.016      0.310      -0.065       0.203
PET.WCRRIUS2.W    -0.0925      0.069     -1.336      0.182      -0.228       0.043
PET.WTTIMUS2.W    -0.0273      0.015     -1.802      0.072      -0.057       0.002
==============================================================================
Omnibus:                      109.495   Durbin-Watson:                   2.145
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              574.138
Skew:                          -0.120   Prob(JB):                    2.13e-125
Kurtosis:                       6.181   Cond. No.                         58.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

As we can see, most of the factors are insignificant

### Apply Neural Network Learning Algorithms

In [189]:
y = np.asarray(df['Last'], dtype = "|S6")
X = df[series_id]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [190]:
scaler = StandardScaler()
#Fit only to the training data
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [191]:
mlp = MLPClassifier(hidden_layer_sizes = (30, 30, 30))

In [192]:
mlp.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [193]:
predictions = mlp.predict(X_test)
print(confusion_matrix(y_test, predictions))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [194]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

  b'-0.000'       0.00      0.00      0.00         3
  b'-0.001'       0.00      0.00      0.00         3
  b'-0.002'       0.00      0.00      0.00         3
  b'-0.003'       0.00      0.00      0.00         6
  b'-0.004'       0.00      0.00      0.00         4
  b'-0.005'       0.00      0.00      0.00         4
  b'-0.006'       0.00      0.00      0.00         3
  b'-0.007'       0.00      0.00      0.00         1
  b'-0.008'       0.00      0.00      0.00         6
  b'-0.009'       0.00      0.00      0.00         3
  b'-0.010'       0.00      0.00      0.00         2
  b'-0.011'       0.00      0.00      0.00         1
  b'-0.012'       0.00      0.00      0.00         1
  b'-0.013'       0.00      0.00      0.00         2
  b'-0.014'       0.50      0.50      0.50         2
  b'-0.015'       0.00      0.00      0.00         5
  b'-0.016'       0.00      0.00      0.00         2
  b'-0.017'       0.00      0.00      0.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [195]:
y_test[0:10]

array([b'-0.092', b'-0.046', b'0.0067', b'0.0610', b'-0.008', b'0.0108',
       b'-0.065', b'0.0188', b'-0.099', b'0.0111'], dtype='|S6')

In [196]:
predictions[0:10]

array([b'0.0313', b'0.0468', b'0.0286', b'-0.002', b'0.0174', b'0.0220',
       b'0.0209', b'-0.172', b'0.0025', b'0.0323'], dtype='|S9')